In [1]:
# Cargar imagen y preprocesar (asegúrate que sea escala de grises)
img = cv2.imread('paciente_neumonia.jpeg', 0)

# 1. Sacar tus caracteristicas
feats_contorno = extraer_descriptores_contorno(img) # Devuelve 4 valores
feats_gabor = extraer_descriptores_gabor(img)       # Devuelve 16 valores (4 angulos * 2 sigmas * 2 metricas)

# 2. Unir todo
vector_caracteristicas_completo = feats_contorno + feats_gabor
# Ahora este vector se guarda en el Excel/CSV para entrenar el modelo

NameError: name 'cv2' is not defined